In [66]:
import numpy as np

In [35]:
class n_queens:
    board = []
    n = None
    queens = None
    
    def __init__(self,n,board,queens = []):
        self.board = board
        self.n = n
        self.queens = queens
     
    def getDiagonals(self,pos):
        
        row,col = pos
        diag = []
        
        for i,j in zip(range(row,self.n),range(col,self.n)):
            if (i,j) != pos:
                diag.append((i,j))
        
        for i,j in zip(range(row,-1,-1),range(col,-1,-1)):
            if (i,j) != pos:
                diag.append((i,j))
                
        for i,j in zip(range(row,-1,-1),range(col,self.n)):
            if (i,j) != pos:
                diag.append((i,j))
       
        for i,j in zip(range(row,self.n),range(col,-1,-1)):
            if (i,j) != pos:
                diag.append((i,j))
        
        diag = sorted(diag,key = lambda x : x[0])
        return diag   
    
    def getRows(self,queen):
        rows = []
        row = queen[0]
        for i in range(self.n):
            rows.append((row,i))
        return rows

    def getColumns(self,queen):
        cols = []
        col = queen[1]
        for i in range(self.n):
            cols.append((i,col))
        
        return cols
    
    def isSafe(self,queen):

        
        for i in self.getRows(queen):
            if self.board[i] == 1 and i != queen:
                return False
        
        for i in self.getColumns(queen):
            if self.board[i] == 1 and i != queen:
                return False
        
        for i in self.getDiagonals(queen):
            if self.board[i] == 1 and i != queen:
                return False
        return True
    
    def isGoal(self):
        
        if len(self.queens) == self.n:
            for queen in self.queens:
                if not self.isSafe(queen):

                    return False
            return True
        
    def isValid(self):
        for queen in self.queens:
            if not self.isSafe(queen):
                return False
        return True

In [36]:
class BackTracking(n_queens):
    neighbours = None
    
    def __init__(self,n,board,queens = []):
        n_queens.__init__(self,n,board,queens)
        #return None
    
    def getNeighbours(self):
        self.neighbours = []
        for i in range(self.n):
            for j in range(self.n):
                if not (i,j) in self.queens:
                    self.neighbours.append((i,j))


In [67]:
def executeBack(node):

    if not node.isValid():
        return False
    
    if node.isGoal():
        return node
    
    
    node.getNeighbours()
    
    for new in node.neighbours:
            board = node.board.copy()
            queens = node.queens[:]
            
            board[new] = 1
            queens.append(new)
            newobj = BackTracking(node.n,board,queens)
            
            final_node = executeBack(newobj)
            
            if final_node:
                return final_node
    return False

        

n = 4   
back = BackTracking(n,np.zeros((n,n)))
p = executeBack(back)
if p:
    print p.board


[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]


In [156]:
class MRV(BackTracking):
    remaining = None
    
    def __init__(self,n,board,queens = [],remaining = []):
        BackTracking.__init__(self,n,board,queens)
        self.remaining = None

    
    def getRemainingValues(self,pos):
        #Obtiene los valores que quedan
        #Tambien considera las reinas
        remaining = []
        moves = []
        for queen in self.queens:
            moves += self.getRows(queen) + self.getColumns(queen) +self.getDiagonals(queen)

        moves += self.getRows(pos) + self.getColumns(pos) +self.getDiagonals(pos)

        for i in range(self.n):
            for j in range(self.n):
                if not (i,j) in moves:
                    remaining.append((i,j))
        return remaining            


    def getNeighbours(self):
        self.neighbours = []

        for i in range(self.n):
            for j in range(self.n):
                if not (i,j) in self.queens:
                    board = self.board.copy()
                    queens = self.queens[:]
                    queens.append((i,j))
                    board[i,j] = 1
                    self.neighbours.append(((i,j),self.getRemainingValues((i,j)),board,queens))
        self.neighbours = sorted(self.neighbours,key = lambda n: len(n[1]))
n = 5
p = MRV(n,np.zeros((n,n)))
#p.queens.append((0,0))
#p.board[0,1] = 1


In [157]:
def executeMRV(node):
    
    if not node.isValid():
        return False
    
    if node.isGoal():
        return node
    
    node.getNeighbours()

    for new in node.neighbours:
        
        newobj = MRV(node.n,new[2],new[3])
        final = executeMRV(newobj)
        
        if final:
            return final
    

print executeMRV(p).board
    

[[0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [208]:
class Degree(BackTracking):
    constrataints = None
    
    def __init__(self,n,board,queens = [],constrataints = []):
        BackTracking.__init__(self,n,board,queens)
        self.constrataints = constrataints
        
    def getConstrataints(self):
        
        moves = []
        for queen in self.queens:
            moves += self.getRows(queen) + self.getColumns(queen) +self.getDiagonals(queen)
        self.constrataints = moves
        
    def getNeighbourConstrataint(self,queen):
        constrataints = self.constrataints[:]
        constrataints += self.getRows(queen) + self.getColumns(queen) +self.getDiagonals(queen)
        return len(constrataints)
    
    def getNeighbours(self):
        self.neighbours = []
        self.getConstrataints()
        
        for i in range(self.n):
            for j in range(self.n):
                if not (i,j) in self.constrataints:
                    board = self.board.copy()
                    queens = self.queens[:]
                    
                    board[i,j] = 1
                    queens.append((i,j))
                    self.neighbours.append((board,queens,self.constrataints,self.getNeighbourConstrataint((i,j))))
        
        
        
        
        self.neighbours = sorted(self.neighbours,key = lambda x: x[3], reverse = True)
                    
        
        
        

p = Degree(n,np.zeros((n,n)))


In [213]:
def executeDegree(node):
    
    if not node.isValid():
        return False
    
    if node.isGoal():
        return node
    
    node.getNeighbours()
    
    for new in node.neighbours:
        
        newobj = Degree(node.n,new[0],new[1],new[2])
        final = executeDegree(newobj)
        
        if final:
            return final

ans = executeDegree(p)
print ans.board

[[0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [218]:
class LCV(Degree):
    
    def __init__(self,n,board,queens = [],constrataints = []):
        Degree.__init__(self,n,board,queens,constrataints)
        
    def getNeighbourConstrataint(self,queen):
        constrataints = self.constrataints[:]
        constrataints += self.getRows(queen) + self.getColumns(queen) +self.getDiagonals(queen)
        return constrataints
        
    
    def getNeighbours(self):
        self.neighbours = []
        self.getConstrataints()
        
        for i in range(self.n):
            for j in range(self.n):
                if not (i,j) in self.constrataints:
                    board = self.board.copy()
                    queens = self.queens[:]
                    
                    board[i,j] = 1
                    queens.append((i,j))
                    self.neighbours.append((board,queens,self.getNeighbourConstrataint((i,j))))
        
        self.neighbours = sorted(self.neighbours,key = lambda x: len(x[2]))

In [225]:
def executeLCV(node):
    
    if not node.isValid():
        return False
    
    if node.isGoal():
        return node
    
    node.getNeighbours()
    
    for new in node.neighbours:
        #print new
        newobj = LCV(node.n,new[0],new[1],new[2])
        final = executeLCV(newobj)
        
        if final:
            return final
p = LCV(n,np.zeros((n,n)))
ans = executeLCV(p)
print ans.board

[[1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]
